<a href="https://colab.research.google.com/github/fabian-castano/timetabling_upb/blob/transition-to-gurobi/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Explore file

In [2]:
# enumerate excel sheets
horas_curso=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='horas_curso')
horas_curso.columns=horas_curso.columns.str.strip()
horas_curso=horas_curso[['ASIGNATURA','codigo', 'horas', 'cantidad de cursos', '# estudiantes',
       'Horas preparacion de clase']]
horas_curso.columns=['ASIGNATURA', 'codigo','horas', 'cantidad_de_cursos', 'total_estudiantes','horas_no_instruccion']
horas_curso.dropna(subset=['codigo'], inplace=True)
horas_curso['codigo']=horas_curso['codigo'].apply(lambda x: x[:9])
#horas_curso['ASIGNATURA']=horas_curso['ASIGNATURA'].apply(lambda x: x[10:])
dict_horas={i['codigo']:i for i in horas_curso.to_dict('records')}

#Save to json file for later use
json.dump(dict_horas, open('data/horas_curso.json', 'w'), indent=4)


In [3]:
idoneidad=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='docente_curso')


# remove final space from column names
idoneidad.columns = idoneidad.columns.str.strip()
idoneidad.rename(columns={'ASIGNATURA ':'ASIGNATURA'}, inplace=True)

display(idoneidad.head())

idoneidad["ASIGNATURA"].fillna(method="ffill",inplace=True)
idoneidad["codigo"]=idoneidad["codigo"].apply(lambda x: x[:9])
idoneidad["ASIGNATURA"]=idoneidad["ASIGNATURA"].apply(lambda x: x[10:])
# replaces all zeros with NaN
idoneidad.replace(0, np.nan, inplace=True)

cols=idoneidad.columns
cols=list(cols)[1:]


idoneidad=idoneidad.melt(id_vars=['codigo', 'ASIGNATURA'], value_vars=cols, var_name='docente', value_name='idoneidad').dropna(subset=['idoneidad'])
idoneidad['idoneidad']=idoneidad.apply(lambda x: (x['codigo'], x['idoneidad']), axis=1)
idoneidad.dropna(subset=['docente'], inplace=True)
idoneidad['docente']=idoneidad.docente.astype(int)
idoneidad=idoneidad.groupby('docente').agg(list)[['idoneidad']].reset_index()
idoneidad['idoneidad']=idoneidad['idoneidad'].apply(lambda x: {i[0]:i[1] for i in x})
idoneidad={i['docente']:i for i in idoneidad.to_dict('records')}
json.dump(idoneidad, open('data/idoneidad.json', 'w'), indent=4)

,ASIGNATURA,codigo,000326151,000336392,000183645,000274691,000259554,000331121,000219830,000187061,000277461,000170520,000226130,000182876,000404903,000234541,000437130,000284431,000182886,000335859,NaN,000228837,000024131,000176738,000120461
0,Introducción_Ingeniería_Industrial,8830-0001,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
1,Introducción_Ingeniería_Industrial,8830-0001,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Introducción_Ingeniería_Industrial,8830-0001,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fundamentos_dibujo_CAD,2027-0059,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0
4,Fundamentos_dibujo_CAD,2027-0059,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
labor_instruccional=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='labor_instruccional')
labor_instruccional.columns=labor_instruccional.columns.str.strip()
dict_labor={i['ID']:i for i in labor_instruccional.to_dict('records')}
json.dump(dict_labor, open('data/labor_instruccional.json', 'w'), indent=4)

# Horas reglamentarias

In [5]:
horas_reglamento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='max_horas')
horas_reglamento.columns=horas_reglamento.columns.str.strip()
dict_horas_reglamento={i['n_cursos']:i for i in horas_reglamento.to_dict('records')}
json.dump(dict_horas_reglamento, open('data/horas_reglamento.json', 'w'), indent=4)

# cursos - requerimientos - tipo_sala

In [6]:
grupo_requerimiento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='grupo_requerimiento')
grupo_requerimiento.columns=grupo_requerimiento.columns.str.strip()
grupo_requerimiento.dropna(subset=['codigo'], inplace=True)
grupo_requerimiento.rename(columns={'cursos':'nombre'}, inplace=True)
grupo_requerimiento={i['codigo']:i for i in grupo_requerimiento.to_dict('records')}
json.dump(grupo_requerimiento, open('data/grupo_requerimiento.json', 'w'), indent=4)

In [7]:
grupo_requerimiento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='grupo_requerimiento')
grupo_requerimiento.columns=grupo_requerimiento.columns.str.strip()

grupo_requerimiento.dropna(subset=['codigo'], inplace=True)
dict_curso_codigo=grupo_requerimiento.set_index('cursos').to_dict()['codigo']


In [8]:
ubicacion_semestral=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='ubicacion_semestral')
ubicacion_semestral.columns=ubicacion_semestral.columns.str.strip()
ubicacion_semestral={i['codigo']:i for i in ubicacion_semestral.to_dict('records')}
json.dump(ubicacion_semestral, open('data/ubicacion_semestral.json', 'w'), indent=4)

# cursos - requerimientos - tipo_sala

In [9]:
capacidad_sala=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='Capac_salon ')
capacidad_sala.columns=capacidad_sala.columns.str.strip()
capacidad_sala={i['aula']:i for i in capacidad_sala.to_dict('records')}
json.dump(capacidad_sala, open('data/capacidad_sala.json', 'w'), indent=4)

In [10]:
df_disp=pd.read_csv('/Users/user/Documents/timetabling_upb/disponibilidad_docente.csv', encoding='ISO-8859-1')
df_disp['ID']=df_disp['ID'].apply(lambda x: str(x).zfill(6))
df_disp

cols=df_disp.columns
cols=list(cols)[2:]

df_disp['available']=df_disp.apply(lambda x: [i for i in cols if x[i]==1], axis=1)
df_disp.drop(columns=cols, inplace=True)
df_disp
days=['L','M','W','J','V','S']
day_len=17
map_slots={str(days[i])+str(j):i*day_len+j for i in range(len(days)) for j in range(day_len)}
df_disp['available']=df_disp['available'].apply(lambda x: [map_slots[i] for i in x])

json.dump(df_disp.set_index('ID').to_dict()['available'], open('data/disponibilidad.json', 'w'), indent=4)